The pre processor is used to process all data into a suitable format for the autoencoder.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def process_data(data, date_cols, output_path):
    """
    Preprocesses the input data and writes the result to a CSV file.

    Parameters:
    data (DataFrame): Input data to be preprocessed.
    date_cols (list): List of date columns to be transformed.
    output_path (str): Path for the processed data CSV file.

    Returns:
    DataFrame: The preprocessed data.
    """

    # Handling DateTime Variables
    for col in date_cols:
        data[col] = pd.to_datetime(data[col], errors='coerce')  # Convert to datetime
        data[f'{col}Year'] = data[col].dt.year  # Extract year
        data[f'{col}Month'] = data[col].dt.month  # Extract month
        data[f'{col}Day'] = data[col].dt.day  # Extract day
        data = data.drop(columns=[col])  # Drop the original date column

    # Automatically identify numerical and categorical variables
    numerical_vars = [col for col in data.select_dtypes(include=[np.number]).columns.tolist() 
                      if not (col.endswith('Year') or col.endswith('Month') or col.endswith('Day'))]
    categorical_vars = data.select_dtypes(include=[object]).columns.tolist()

    # Print information for verification
    print(f"Numerical Columns: {numerical_vars}")
    print(f"Categorical Columns: {categorical_vars}")

    # Create transformers
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Initialize ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_vars),
            ('cat', categorical_transformer, categorical_vars)
        ]
    )

    # Fit and transform the data
    data_preprocessed = preprocessor.fit_transform(data)

    # Get feature names after one-hot encoding
    onehot_columns = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_vars)

    # Combine the feature names
    feature_names = numerical_vars + onehot_columns.tolist()

    # Print and verify the lengths
    print(f"Data Shape after Preprocessing: {data_preprocessed.shape}")
    print(f"Number of Feature Names: {len(feature_names)}")

    data_preprocessed_dense = data_preprocessed.toarray() if hasattr(data_preprocessed, 'toarray') else data_preprocessed
    data_preprocessed_df = pd.DataFrame(data_preprocessed_dense, columns=feature_names)

    # Save preprocessed data to CSV
    data_preprocessed_df.to_csv(output_path, index=False)
    return data_preprocessed_df

# MODEL TRAINING DATA
your_path = r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima'
file_name1 = '\CLEAN'
file_path1 = your_path + file_name1 + '.csv'
date_cols = ['effectiveDate', 'eventDateTime', 'executionDateTime', 'expirationDate', 'sDRreceiptTimestamp']
data1 = pd.read_csv(file_path)

output_path1 = your_path + file_name1 + '_Processed.csv'
processed_data_clean = process_data(data1, date_cols, output_path1)

# TRADE DATA
file_name2 = '\TRADES'
file_path2 = your_path + file_name2 + '.csv'
data2 = pd.read_csv(file_path2)
output_path2 = your_path + file_name2 + '_Processed.csv'
processed_data_trade = process_data(data2, date_cols, output_path2)


THE CODE BELOW WORKS

In [48]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def process_data(data, date_cols, output_path, train_feature_names=None):
    """
    Preprocesses the input data and writes the result to a CSV file.

    Parameters:
    data (DataFrame): Input data to be preprocessed.
    date_cols (list): List of date columns to be transformed.
    output_path (str): Path for the processed data CSV file.
    train_feature_names (list): Feature names from the training data.

    Returns:
    DataFrame: The preprocessed data.
    """

    # Handling DateTime Variables
    for col in date_cols:
        data[col] = pd.to_datetime(data[col], errors='coerce')  # Convert to datetime
        data[f'{col}Year'] = data[col].dt.year  # Extract year
        data[f'{col}Month'] = data[col].dt.month  # Extract month
        data[f'{col}Day'] = data[col].dt.day  # Extract day
        data = data.drop(columns=[col])  # Drop the original date column

    # Automatically identify numerical and categorical variables
    numerical_vars = [col for col in data.select_dtypes(include=[np.number]).columns.tolist() 
                      if not (col.endswith('Year') or col.endswith('Month') or col.endswith('Day'))]
    categorical_vars = data.select_dtypes(include=[object]).columns.tolist()

    # Print information for verification
    print(f"Numerical Columns: {numerical_vars}")
    print(f"Categorical Columns: {categorical_vars}")

    # Create transformers
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Initialize ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_vars),
            ('cat', categorical_transformer, categorical_vars)
        ]
    )

    # Fit and transform the data
    data_preprocessed = preprocessor.fit_transform(data)

    # Get feature names after one-hot encoding
    onehot_columns = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_vars)

    # Combine the feature names
    feature_names = numerical_vars + onehot_columns.tolist()

    # If the function was provided training feature names, add any missing columns with zeros
    if train_feature_names is not None:
        missing_columns = set(train_feature_names) - set(feature_names)
        for missing_col in missing_columns:
            data_preprocessed = np.column_stack((data_preprocessed, np.zeros(data_preprocessed.shape[0])))
        feature_names = train_feature_names

    # Print and verify the lengths
    print(f"Data Shape after Preprocessing: {data_preprocessed.shape}")
    print(f"Number of Feature Names: {len(feature_names)}")

    data_preprocessed_dense = data_preprocessed.toarray() if hasattr(data_preprocessed, 'toarray') else data_preprocessed
    data_preprocessed_df = pd.DataFrame(data_preprocessed_dense, columns=feature_names)

    # Handle columns with all missing values
    for col in data_preprocessed_df.columns:
        if data_preprocessed_df[col].isnull().all():
            data_preprocessed_df[col].fillna(0, inplace=True)  # fill NaN with 0 for these particular columns

    # Save preprocessed data to CSV
    data_preprocessed_df.to_csv(output_path, index=False)
    return data_preprocessed_df

# SET THIS TO THE FOLDER WHERE THE FILES ARE LOCATED
your_path = r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima'

# For training data
file_name1 = '\CLEAN'
file_path1 = your_path + file_name1 + '.csv'
date_cols = ['effectiveDate', 'eventDateTime', 'executionDateTime', 'expirationDate', 'sDRreceiptTimestamp']
data1 = pd.read_csv(file_path1)
output_path1 = your_path + file_name1 + '_Processed.csv'
processed_data_clean = process_data(data1, date_cols, output_path1)

# For trade data
file_name2 = '\TRADES'
file_path2 = your_path + file_name2 + '.csv'
data2 = pd.read_csv(file_path2)
output_path2 = your_path + file_name2 + '_Processed.csv'
processed_data_trade = process_data(data2, date_cols, output_path2, processed_data_clean.columns.tolist())

Numerical Columns: ['leg1NotionalAmount', 'leg2NotionalAmount', 'leg2ResetFrequencyMultiplier', 'leg1FixedRate', 'leg1FixedRatePaymentFrequencyMultiplier', 'leg2FloatingRatePaymentFrequencyMultiplier', 'leg2UnderlierTenorMultiplier']
Categorical Columns: ['action', 'assetClass', 'blockTradeIndicator', 'cleared', 'deliveryType', 'disseminationTimestamp', 'event', 'instrumentType', 'leg1FixedRateDayCount', 'leg1FloatingRateDayCount', 'leg1NotionalCurrency', 'leg1NotionalScheduleType', 'leg1SettlementCurrency', 'leg1UnderlyingAssetOrContractType', 'leg2FloatingRateDayCount', 'leg2NotionalCurrency', 'leg2NotionalScheduleType', 'leg2SettlementCurrency', 'nonStandardTermIndicator', 'platformID', 'primeBrokerageTransactionIndicator', 'productName', 'leg1FixedRatePaymentFrequencyPeriod', 'leg2FloatingRatePaymentFrequencyPeriod', 'leg2UnderlierCurrency', 'leg2UnderlierID', 'leg2UnderlierIDSource', 'leg2UnderlierTenorPeriod', 'leg2ResetFrequencyPeriod']
Data Shape after Preprocessing: (5804, 504

c:\Users\gusta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: ['blockTradeIndicator' 'nonStandardTermIndicator' 'platformID'
 'primeBrokerageTransactionIndicator']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def process_data(data, date_cols, output_path, train_feature_names=None):
    """
    Preprocesses the input data and writes the result to a CSV file.
    
    Parameters:
    data (DataFrame): Input data to be preprocessed.
    date_cols (list): List of date columns to be transformed.
    output_path (str): Path for the processed data CSV file.
    train_feature_names (list): Feature names from the training data.
    
    Returns:
    DataFrame: The preprocessed data.
    """

    for col in date_cols:
        data[col] = pd.to_datetime(data[col], errors='coerce')
        data[f'{col}Year'] = data[col].dt.year
        data[f'{col}Month'] = data[col].dt.month
        data[f'{col}Day'] = data[col].dt.day
        data = data.drop(columns=[col])

    numerical_vars = [col for col in data.select_dtypes(include=[np.number]).columns.tolist() 
                      if not (col.endswith('Year') or col.endswith('Month') or col.endswith('Day'))]
    categorical_vars = data.select_dtypes(include=[object]).columns.tolist()

    print(f"Numerical Columns: {numerical_vars}")
    print(f"Categorical Columns: {categorical_vars}")

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_vars),
            ('cat', categorical_transformer, categorical_vars)
        ]
    )

    data_preprocessed = preprocessor.fit_transform(data)
    onehot_columns = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_vars)
    feature_names = numerical_vars + onehot_columns.tolist()

    print(f"Data Shape after Preprocessing: {data_preprocessed.shape}")
    print(f"Number of Feature Names: {len(feature_names)}")

    data_preprocessed_dense = data_preprocessed.toarray() if hasattr(data_preprocessed, 'toarray') else data_preprocessed
    data_preprocessed_df = pd.DataFrame(data_preprocessed_dense, columns=feature_names)

    if train_feature_names is not None:
        missing_cols = set(train_feature_names) - set(data_preprocessed_df.columns)
        for c in missing_cols:
            data_preprocessed_df[c] = 0
        data_preprocessed_df = data_preprocessed_df[train_feature_names]

    data_preprocessed_df.to_csv(output_path, index=False)
    return data_preprocessed_df

# Usage example
your_path = r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima'

# For training data
file_name1 = '\CLEAN'
file_path1 = your_path + file_name1 + '.csv'
date_cols = ['effectiveDate', 'eventDateTime', 'executionDateTime', 'expirationDate', 'sDRreceiptTimestamp']
data1 = pd.read_csv(file_path1)
output_path1 = your_path + file_name1 + '_Processed.csv'
processed_data_clean = process_data(data1, date_cols, output_path1)

# For trade data
file_name2 = '\TRADES'
file_path2 = your_path + file_name2 + '.csv'
data2 = pd.read_csv(file_path2)
output_path2 = your_path + file_name2 + '_Processed.csv'
processed_data_trade = process_data(data2, date_cols, output_path2, processed_data_clean.columns.tolist())


Numerical Columns: ['leg1NotionalAmount', 'leg2NotionalAmount', 'leg2ResetFrequencyMultiplier', 'leg1FixedRate', 'leg1FixedRatePaymentFrequencyMultiplier', 'leg2FloatingRatePaymentFrequencyMultiplier', 'leg2UnderlierTenorMultiplier']
Categorical Columns: ['action', 'assetClass', 'blockTradeIndicator', 'cleared', 'deliveryType', 'disseminationTimestamp', 'event', 'instrumentType', 'leg1FixedRateDayCount', 'leg1FloatingRateDayCount', 'leg1NotionalCurrency', 'leg1NotionalScheduleType', 'leg1SettlementCurrency', 'leg1UnderlyingAssetOrContractType', 'leg2FloatingRateDayCount', 'leg2NotionalCurrency', 'leg2NotionalScheduleType', 'leg2SettlementCurrency', 'nonStandardTermIndicator', 'platformID', 'primeBrokerageTransactionIndicator', 'productName', 'leg1FixedRatePaymentFrequencyPeriod', 'leg2FloatingRatePaymentFrequencyPeriod', 'leg2UnderlierCurrency', 'leg2UnderlierID', 'leg2UnderlierIDSource', 'leg2UnderlierTenorPeriod', 'leg2ResetFrequencyPeriod']
Data Shape after Preprocessing: (5804, 504

c:\Users\gusta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: ['blockTradeIndicator' 'nonStandardTermIndicator' 'platformID'
 'primeBrokerageTransactionIndicator']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


ValueError: Shape of passed values is (20, 96), indices imply (20, 100)

In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load your data
your_path = r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima'
file_name = '\TRADES'
file_path = your_path + file_name + '.csv'
data = pd.read_csv(file_path)

# This is currently the date columns for the IRS file and may need to be changed for e.g. Commodity etc. 
date_cols = ['effectiveDate', 'eventDateTime', 'executionDateTime', 'expirationDate', 'sDRreceiptTimestamp']


# Handling DateTime Variables
for col in date_cols:
    data[col] = pd.to_datetime(data[col], errors='coerce')  # Convert to datetime
    data[f'{col}Year'] = data[col].dt.year  # Extract year
    data[f'{col}Month'] = data[col].dt.month  # Extract month
    data[f'{col}Day'] = data[col].dt.day  # Extract day
    data = data.drop(columns=[col])  # Drop the original date column

# Automatically identify numerical and categorical variables
numerical_vars = [col for col in data.select_dtypes(include=[np.number]).columns.tolist() 
                  if not (col.endswith('Year') or col.endswith('Month') or col.endswith('Day'))]
categorical_vars = data.select_dtypes(include=[object]).columns.tolist()

# Print information for verification
print(f"Numerical Columns: {numerical_vars}")
print(f"Categorical Columns: {categorical_vars}")

# Create transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Initialize ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_vars),
        ('cat', categorical_transformer, categorical_vars)
    ]
)

# Fit and transform the data
data_preprocessed = preprocessor.fit_transform(data)

# Get feature names after one-hot encoding
onehot_columns = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_vars)

# Combine the feature names
feature_names = numerical_vars + onehot_columns.tolist()

# Print and verify the lengths
print(f"Data Shape after Preprocessing: {data_preprocessed.shape}")
print(f"Number of Feature Names: {len(feature_names)}")


data_preprocessed_dense = data_preprocessed.toarray() if hasattr(data_preprocessed, 'toarray') else data_preprocessed
data_preprocessed_df = pd.DataFrame(data_preprocessed_dense, columns=feature_names)


output_path = your_path + file_name + '_Processed.csv'
data_preprocessed_df.to_csv(output_path, index=False)



Numerical Columns: ['blockTradeIndicator', 'leg1NotionalAmount', 'leg2NotionalAmount', 'leg2ResetFrequencyMultiplier', 'nonStandardTermIndicator', 'platformID', 'primeBrokerageTransactionIndicator', 'leg1FixedRate', 'leg1FixedRatePaymentFrequencyMultiplier', 'leg2FloatingRatePaymentFrequencyMultiplier', 'leg2UnderlierTenorMultiplier']
Categorical Columns: ['action', 'assetClass', 'cleared', 'deliveryType', 'disseminationTimestamp', 'event', 'instrumentType', 'leg1FixedRateDayCount', 'leg1FloatingRateDayCount', 'leg1NotionalCurrency', 'leg1NotionalScheduleType', 'leg1SettlementCurrency', 'leg1UnderlyingAssetOrContractType', 'leg2FloatingRateDayCount', 'leg2NotionalCurrency', 'leg2NotionalScheduleType', 'leg2SettlementCurrency', 'productName', 'leg1FixedRatePaymentFrequencyPeriod', 'leg2FloatingRatePaymentFrequencyPeriod', 'leg2UnderlierCurrency', 'leg2UnderlierID', 'leg2UnderlierIDSource', 'leg2UnderlierTenorPeriod', 'leg2ResetFrequencyPeriod']
Data Shape after Preprocessing: (20, 96)
N

c:\Users\gusta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: ['blockTradeIndicator' 'nonStandardTermIndicator' 'platformID'
 'primeBrokerageTransactionIndicator']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


ValueError: Shape of passed values is (20, 96), indices imply (20, 100)

In [11]:
import os

# Define the path
path = your_path

# Check if the path does not exist
if not os.path.exists(path):
    # Create the directory
    os.makedirs(path)

# Now you can save the DataFrame
data_preprocessed.to_csv(f'{path}/processed_data.csv', index=False)



AttributeError: 'csr_matrix' object has no attribute 'to_csv'